# RWA density analytics

In this notebook I aim at creating a simple analytical app to calculate and visualize RWA density to help explain its dynamics - with python and [atoti](https://www.atoti.io).


See for example, in this recording on the summary screen we can identify that for the iBank CIB the RWA density went up materially. Then in a Pivot table we can explore the contributors of the density calculation and identify the main drivers.

<img src=./app-preview.gif/ width = 90% margin = auto>

## Background

See for example, how the RWA density term is introduced in the [BIS working paper "Leverage and Risk Weighted Capital Requirements"](https://www.bis.org/publ/work586.pdf):
    
$\text{RWA density} = \frac{RWA}{Exposure}$

where _Exposures_ is defined as "The total exposure is given by total assets and other commitments. A detailed explanation of the definition of capital and total exposures can be found in BCBS (2014a)", then RWA density becomes:

$\text{RWA density} = \frac{RWA}{TotalAssets}$

RWA density gives a perspective on a bank's overall riskiness and is used by stakeholders (investors, regulators) to compare financial organizations risk profiles, however as identified by the authors in [COMPARING RISK-WEIGHTED ASSETS: THE IMPORTANCE OF SUPERVISORY VALIDATION PROCESSES](https://www.researchgate.net/profile/Raul_Garcia21/publication/292140152_Comparing_risk-weighted_assets_The_importance_of_the_supervisory_validation_processes/links/56a9e84508ae7f592f0e487b/Comparing-risk-weighted-assets-The-importance-of-the-supervisory-validation-processes.pdf):

> The flaws exposed in this ratio have raised doubts over its usefulness as an analytical tool: in particular, its intrinsic inconsistency and the fact that, by attempting to cover all the bank’s activity, its value is influenced by a multitude of different factors whose effect is difficult to assess (balance sheet structure, investment policies, type of business, operations and also the calculation methodologies for RWA). Indeed, it is likely that their use will readily lead to mistaken conclusions. If the aim is to detect methodological or criteria-related inconsistencies in the calculation of RWA across banks, this ratio does not appear to be overly useful.

Banks report the RWA density as part of their Pillar 3 disclosures, for example:

- [CIBC Q3 2020](https://www.cibc.com/content/dam/about_cibc/investor_relations/pdfs/quarterly_results/2020/q320disclosure-en.pdf)
- [Sberbank 2018](https://www.sberbank.com/common/img/uploaded/files/info/en/4482-2018_eng.pdf)
- [Nordea 2018](https://www.nordea.com/Images/35-305679/nordea-capital-and-risk-management-report-2018-updated-version.pdf)

The RWA density shows a range of variation that fluctuates from 15% to almost 60% for European international IRB banks

# Risk data aggregation and reporting capabilities for RWA density

We create a prototype to visualize RWA density and its inputs across reporting periods, evalute changes and identify main drivers.

My sample data contains RWA and exposures allocated to transactions. 
In a more advanced setup, RWA could be computed according to the appropriate methodology (for example, FRTB for market risk or IRB for credit risk) this way allowing for a more sophisticated analysis.

![RWA flow](RWA-app.png)


# Random sample data - positions Exposures and RWAs for 12 month ends

In [25]:
#!python3 sample_data_generator.py

In [26]:
sample_data = pd.read_csv("sample_data.csv")
sample_data.columns

Index(['asset_classes', 'products', 'maturity_buckets', 'business_units',
       'pd_bucket', 'legal_entities', 'transaction_id', 'exposure', 'RWA',
       'reporting_date'],
      dtype='object')

In [27]:
sample_data.shape

(12100, 10)

# launch atoti

In [28]:
import atoti as tt
from atoti.config import create_config

# launching atoti cube and ui
config = create_config(metadata_db="./metadata.db", sampling_mode=tt.sampling.FULL)
session = tt.create_session(config=config)


# loading data into atoti data store
exposures_datastore = session.read_pandas(
    sample_data,
    store_name="exposures",
    keys=["reporting_date", "transaction_id"],
)

# populating measures and dimensions in the cube
cube = session.create_cube(exposures_datastore, "RWA density", mode="no_measures")

Welcome to atoti 0.5.1!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


'http://localhost:63031'

# Create measures for RWA density analytics

In [29]:
# In my example RWA is assumed to be additive across input data. This can be replaced with an actual calculation
cube.measures["RWA"] = tt.agg.sum(exposures_datastore["RWA"])
cube.measures["exposure"] = tt.agg.sum(exposures_datastore["exposure"])
cube.measures["RWA density"] = cube.measures["RWA"] / cube.measures["exposure"]
cube.measures["RWA density"].formatter = "DOUBLE[0.00%]"


cube.measures["Previous RWA density"] = tt.shift(
    cube.measures["RWA density"], cube.levels["reporting_date"], offset=1
)
up_or_down = tt.where(
    cube.measures["RWA density"] > cube.measures["Previous RWA density"], "Up", "Down"
)
cube.measures["Up or Down"] = tt.where(
    cube.measures["Previous RWA density"] != None, up_or_down, "New"
)
cube.measures["RWA density Chg"] = tt.where(
    cube.measures["Previous RWA density"] != None,
    cube.measures["RWA density"] - cube.measures["Previous RWA density"],
)
cube.measures["RWA density Chg"].formatter = "DOUBLE[+0.00%;-0.00%]"

cube.hierarchies["reporting_date"].slicing = True
cube.levels["reporting_date"].comparator = tt.comparator.DESC

cube.levels["maturity_buckets"].comparator = tt.comparator.first_members(
    ">1y",
    "1 year ≤ residual maturity < 2 years",
    "2 years ≤ residual maturity < 5 years",
    "5 years ≤ residual maturity < 10 years",
    "> 10 years",
)

cube.levels["pd_bucket"].comparator = tt.comparator.first_members(
    "0.00 to < 0.15",
    "0.15 to < 0.25",
    "0.25 to < 0.50",
    "0.50 to < 0.75",
    "0.75 to < 2.50",
    "2.50 to < 10.00",
    "10.00 to < 100",
    "100 default",
)

cube.measures["Previous RWA"] = tt.shift(
    cube.measures["RWA"], cube.levels["reporting_date"], offset=1
)
cube.measures["Previous exposure"] = tt.shift(
    cube.measures["exposure"], cube.levels["reporting_date"], offset=1
)

cube.measures["RWA Chg"] = tt.where(
    cube.measures["Previous RWA"] != None,
    cube.measures["RWA"] - cube.measures["Previous RWA"],
)
cube.measures["RWA % Chg"] = tt.where(
    cube.measures["Previous RWA"] != None,
    cube.measures["RWA"] / cube.measures["Previous RWA"] - 1,
)
cube.measures["RWA % Chg"].formatter = "DOUBLE[0.00%]"


cube.measures["exposure Chg"] = tt.where(
    cube.measures["Previous exposure"] != None,
    cube.measures["exposure"] - cube.measures["Previous exposure"],
)
cube.measures["exposure % Chg"] = tt.where(
    cube.measures["Previous exposure"] != None,
    cube.measures["exposure"] / cube.measures["Previous exposure"] - 1,
)
cube.measures["exposure % Chg"].formatter = "DOUBLE[0.00%]"

# Build dashboards

In [30]:
session.url

'http://localhost:63031'